## 爬取Pokemon图片

In [54]:
import requests
from bs4 import BeautifulSoup
import re
import os
from IPython.display import Image
import time
import pandas as pd
import random

# url 前缀
url_prefix = 'https://wiki.52poke.com'

**1.获取网页源码**

In [13]:
def crawl(url, url_suffix):
    '''
    根据传入的 url 获取html
    returns:
        html: 获取到的 html
    '''
    url = url + url_suffix
    html = requests.get(url).text
    return html

**2.解析网页**

In [36]:
def parse(html):
    '''
    根据传入的 html 找到我们需要的数据
    return:
        all_pokemon_urls: 所有 Pokemon 的url
    '''
    # 创建 html 实例
    soup = BeautifulSoup(html, 'html.parser')
    
    # 所以所有 href 标签内容
    all_hrefs = soup.find(class_='mw-content-ltr').find_all(class_='mw-redirect')
    
    # 提取所有链接
    all_urls = [url.get('href') for url in all_hrefs]

    # 仅提取格式为 `/wiki/Bulbasaur` 的所有链接
    all_pokemon_urls = []
    # 使用正则表达式匹配想要的链接
    # 但是执行速度比使用下面的while慢了一倍
#     for i in range(len(all_urls)):
#         url = all_urls[i]
#         find = re.search(r'/wiki/\w.*', url)
#         if find:
#             all_pokemon_urls.append(find.group(0)
    i = 0
    while i < len(all_urls):
        if i%2 != 0:
            all_pokemon_urls.append(all_urls[i])
        i += 1
    
    return all_pokemon_urls

**3.获取下一层级：Pokemon 详细信息页面的链接**

In [15]:
now = time.time()

# 获取 Pokemon html
html = crawl(url_prefix, '/wiki/宝可梦列表（按全国图鉴编号）/简单版')
# 获取所有 Pokemon 详细信息链接
all_pokemon_urls = parse(html)

print('爬取完成，共花费 {} 秒'.format(time.time()-now))

爬取完成，共花费 1.771186113357544 秒


In [16]:
all_pokemon_urls

['/wiki/Bulbasaur',
 '/wiki/Ivysaur',
 '/wiki/Venusaur',
 '/wiki/Charmander',
 '/wiki/Charmeleon',
 '/wiki/Charizard',
 '/wiki/Squirtle',
 '/wiki/Wartortle',
 '/wiki/Blastoise',
 '/wiki/Caterpie',
 '/wiki/Metapod',
 '/wiki/Butterfree',
 '/wiki/Weedle',
 '/wiki/Kakuna',
 '/wiki/Beedrill',
 '/wiki/Pidgey',
 '/wiki/Pidgeotto',
 '/wiki/Pidgeot',
 '/wiki/Rattata',
 '/wiki/Raticate',
 '/wiki/Spearow',
 '/wiki/Fearow',
 '/wiki/Ekans',
 '/wiki/Arbok',
 '/wiki/Pikachu',
 '/wiki/Raichu',
 '/wiki/Sandshrew',
 '/wiki/Sandslash',
 '/wiki/Nidoran%E2%99%80',
 '/wiki/Nidorina',
 '/wiki/Nidoqueen',
 '/wiki/Nidoran%E2%99%82',
 '/wiki/Nidorino',
 '/wiki/Nidoking',
 '/wiki/Clefairy',
 '/wiki/Clefable',
 '/wiki/Vulpix',
 '/wiki/Ninetales',
 '/wiki/Jigglypuff',
 '/wiki/Wigglytuff',
 '/wiki/Zubat',
 '/wiki/Golbat',
 '/wiki/Oddish',
 '/wiki/Gloom',
 '/wiki/Vileplume',
 '/wiki/Paras',
 '/wiki/Parasect',
 '/wiki/Venonat',
 '/wiki/Venomoth',
 '/wiki/Diglett',
 '/wiki/Dugtrio',
 '/wiki/Meowth',
 '/wiki/Persian',


**4.根据 `all_pokemon_urls` 的链接，逐一爬取每个 Pokemon 的详细网页中的图片链接**

In [37]:
img_urls = []

now = time.time()

for i in range(len(all_pokemon_urls)):
    # 获取 Pokemon 介绍页面的 html
    html = crawl(url_prefix, all_pokemon_urls[i])
    
    # 解析 html
    soup = BeautifulSoup(html, 'html.parser')
    
    # 获取 image url
    img_tag = soup.find('table', class_='roundy bgwhite fulltable').find('img')
    try:
        # 获取分辨率较高的image链接 600px
        portion_url = img_tag.get('data-srcset').split(',')[-1]
    except AttributeError as err: # 部分 Pokemon 并没有 data-srcset 关键字
#         print('AttributeError: {}'.format(err))
        # 用 data-url 代替获取 image url
        # 不足是分辨率较低 300px
        portion_url = img_tag.get('data-url')
        
    img_url = 'https://' + re.findall(r'media.*png', portion_url)[0]
    img_urls.append(img_url)
#     print('{}. 抓取 {} 完成'.format(i+1, img_url))
    
# 共花费 1499.1186759471893 秒 / 25分钟
print('抓取所有链接完成，共花费 {} 秒'.format(time.time()-now))

1. 抓取 https://media.52poke.com/wiki/thumb/2/21/001Bulbasaur.png/600px-001Bulbasaur.png 完成
2. 抓取 https://media.52poke.com/wiki/thumb/7/73/002Ivysaur.png/600px-002Ivysaur.png 完成
3. 抓取 https://media.52poke.com/wiki/thumb/a/ae/003Venusaur.png/600px-003Venusaur.png 完成
4. 抓取 https://media.52poke.com/wiki/thumb/7/73/004Charmander.png/600px-004Charmander.png 完成
5. 抓取 https://media.52poke.com/wiki/4/4a/005Charmeleon.png 完成
6. 抓取 https://media.52poke.com/wiki/thumb/7/7e/006Charizard.png/600px-006Charizard.png 完成
7. 抓取 https://media.52poke.com/wiki/thumb/3/39/007Squirtle.png/600px-007Squirtle.png 完成
8. 抓取 https://media.52poke.com/wiki/0/0c/008Wartortle.png 完成
9. 抓取 https://media.52poke.com/wiki/thumb/0/02/009Blastoise.png/600px-009Blastoise.png 完成
10. 抓取 https://media.52poke.com/wiki/5/5d/010Caterpie.png 完成
11. 抓取 https://media.52poke.com/wiki/thumb/c/cd/011Metapod.png/600px-011Metapod.png 完成
12. 抓取 https://media.52poke.com/wiki/thumb/d/d1/012Butterfree.png/600px-012Butterfree.png 完成
13. 抓取 https

120. 抓取 https://media.52poke.com/wiki/4/4f/120Staryu.png 完成
121. 抓取 https://media.52poke.com/wiki/c/cd/121Starmie.png 完成
122. 抓取 https://media.52poke.com/wiki/e/ec/122Mr._Mime.png 完成
123. 抓取 https://media.52poke.com/wiki/b/ba/123Scyther.png 完成
124. 抓取 https://media.52poke.com/wiki/7/7c/124Jynx.png 完成
125. 抓取 https://media.52poke.com/wiki/thumb/d/de/125Electabuzz.png/600px-125Electabuzz.png 完成
126. 抓取 https://media.52poke.com/wiki/thumb/8/8c/126Magmar.png/600px-126Magmar.png 完成
127. 抓取 https://media.52poke.com/wiki/thumb/7/71/127Pinsir.png/600px-127Pinsir.png 完成
128. 抓取 https://media.52poke.com/wiki/b/b8/128Tauros.png 完成
129. 抓取 https://media.52poke.com/wiki/0/02/129Magikarp.png 完成
130. 抓取 https://media.52poke.com/wiki/4/41/130Gyarados.png 完成
131. 抓取 https://media.52poke.com/wiki/thumb/a/ab/131Lapras.png/600px-131Lapras.png 完成
132. 抓取 https://media.52poke.com/wiki/3/36/132Ditto.png 完成
133. 抓取 https://media.52poke.com/wiki/thumb/e/e2/133Eevee.png/600px-133Eevee.png 完成
134. 抓取 https://med

223. 抓取 https://media.52poke.com/wiki/thumb/9/95/223Remoraid.png/600px-223Remoraid.png 完成
224. 抓取 https://media.52poke.com/wiki/thumb/c/cb/224Octillery.png/600px-224Octillery.png 完成
225. 抓取 https://media.52poke.com/wiki/thumb/3/3f/225Delibird.png/600px-225Delibird.png 完成
226. 抓取 https://media.52poke.com/wiki/thumb/c/c5/226Mantine.png/600px-226Mantine.png 完成
227. 抓取 https://media.52poke.com/wiki/thumb/3/35/227Skarmory.png/600px-227Skarmory.png 完成
228. 抓取 https://media.52poke.com/wiki/5/53/228Houndour.png 完成
229. 抓取 https://media.52poke.com/wiki/5/51/229Houndoom.png 完成
230. 抓取 https://media.52poke.com/wiki/thumb/3/3c/230Kingdra.png/600px-230Kingdra.png 完成
231. 抓取 https://media.52poke.com/wiki/thumb/d/d3/231Phanpy.png/600px-231Phanpy.png 完成
232. 抓取 https://media.52poke.com/wiki/thumb/5/53/232Donphan.png/600px-232Donphan.png 完成
233. 抓取 https://media.52poke.com/wiki/thumb/9/99/233Porygon2.png/600px-233Porygon2.png 完成
234. 抓取 https://media.52poke.com/wiki/thumb/5/50/234Stantler.png/600px-234

327. 抓取 https://media.52poke.com/wiki/8/8f/327Spinda.png 完成
328. 抓取 https://media.52poke.com/wiki/7/76/328Trapinch.png 完成
329. 抓取 https://media.52poke.com/wiki/a/af/329Vibrava.png 完成
330. 抓取 https://media.52poke.com/wiki/f/f1/330Flygon.png 完成
331. 抓取 https://media.52poke.com/wiki/thumb/1/12/331Cacnea.png/600px-331Cacnea.png 完成
332. 抓取 https://media.52poke.com/wiki/4/41/332Cacturne.png 完成
333. 抓取 https://media.52poke.com/wiki/9/99/333Swablu.png 完成
334. 抓取 https://media.52poke.com/wiki/thumb/d/da/334Altaria.png/600px-334Altaria.png 完成
335. 抓取 https://media.52poke.com/wiki/d/d3/335Zangoose.png 完成
336. 抓取 https://media.52poke.com/wiki/d/d6/336Seviper.png 完成
337. 抓取 https://media.52poke.com/wiki/e/eb/337Lunatone.png 完成
338. 抓取 https://media.52poke.com/wiki/9/90/338Solrock.png 完成
339. 抓取 https://media.52poke.com/wiki/6/60/339Barboach.png 完成
340. 抓取 https://media.52poke.com/wiki/6/60/340Whiscash.png 完成
341. 抓取 https://media.52poke.com/wiki/3/3d/341Corphish.png 完成
342. 抓取 https://media.52poke.

439. 抓取 https://media.52poke.com/wiki/thumb/5/5a/439Mime_Jr..png/600px-439Mime_Jr..png 完成
440. 抓取 https://media.52poke.com/wiki/thumb/2/27/440Happiny.png/600px-440Happiny.png 完成
441. 抓取 https://media.52poke.com/wiki/thumb/b/bf/441Chatot.png/600px-441Chatot.png 完成
442. 抓取 https://media.52poke.com/wiki/8/8e/442Spiritomb.png 完成
443. 抓取 https://media.52poke.com/wiki/6/68/443Gible.png 完成
444. 抓取 https://media.52poke.com/wiki/9/9d/444Gabite.png 完成
445. 抓取 https://media.52poke.com/wiki/thumb/f/fa/445Garchomp.png/600px-445Garchomp.png 完成
446. 抓取 https://media.52poke.com/wiki/thumb/b/b2/446Munchlax.png/600px-446Munchlax.png 完成
447. 抓取 https://media.52poke.com/wiki/thumb/a/a2/447Riolu.png/600px-447Riolu.png 完成
448. 抓取 https://media.52poke.com/wiki/thumb/d/d7/448Lucario.png/600px-448Lucario.png 完成
449. 抓取 https://media.52poke.com/wiki/a/ab/449Hippopotas.png 完成
450. 抓取 https://media.52poke.com/wiki/thumb/5/5f/450Hippowdon.png/600px-450Hippowdon.png 完成
451. 抓取 https://media.52poke.com/wiki/4/47/451

538. 抓取 https://media.52poke.com/wiki/thumb/7/74/538Throh.png/600px-538Throh.png 完成
539. 抓取 https://media.52poke.com/wiki/thumb/a/a8/539Sawk.png/600px-539Sawk.png 完成
540. 抓取 https://media.52poke.com/wiki/thumb/4/4a/540Sewaddle.png/600px-540Sewaddle.png 完成
541. 抓取 https://media.52poke.com/wiki/thumb/2/2b/541Swadloon.png/600px-541Swadloon.png 完成
542. 抓取 https://media.52poke.com/wiki/thumb/8/8e/542Leavanny.png/600px-542Leavanny.png 完成
543. 抓取 https://media.52poke.com/wiki/thumb/0/0e/543Venipede.png/600px-543Venipede.png 完成
544. 抓取 https://media.52poke.com/wiki/thumb/b/bc/544Whirlipede.png/600px-544Whirlipede.png 完成
545. 抓取 https://media.52poke.com/wiki/thumb/c/cb/545Scolipede.png/600px-545Scolipede.png 完成
546. 抓取 https://media.52poke.com/wiki/thumb/4/44/546Cottonee.png/600px-546Cottonee.png 完成
547. 抓取 https://media.52poke.com/wiki/thumb/a/a2/547Whimsicott.png/600px-547Whimsicott.png 完成
548. 抓取 https://media.52poke.com/wiki/thumb/0/0b/548Petilil.png/600px-548Petilil.png 完成
549. 抓取 https://

638. 抓取 https://media.52poke.com/wiki/thumb/6/65/638Cobalion.png/600px-638Cobalion.png 完成
639. 抓取 https://media.52poke.com/wiki/thumb/a/ad/639Terrakion.png/600px-639Terrakion.png 完成
640. 抓取 https://media.52poke.com/wiki/thumb/7/79/640Virizion.png/600px-640Virizion.png 完成
641. 抓取 https://media.52poke.com/wiki/thumb/0/08/641Tornadus.png/600px-641Tornadus.png 完成
642. 抓取 https://media.52poke.com/wiki/thumb/b/b8/642Thundurus.png/600px-642Thundurus.png 完成
643. 抓取 https://media.52poke.com/wiki/thumb/8/8d/643Reshiram.png/600px-643Reshiram.png 完成
644. 抓取 https://media.52poke.com/wiki/thumb/8/81/644Zekrom.png/600px-644Zekrom.png 完成
645. 抓取 https://media.52poke.com/wiki/thumb/b/bb/645Landorus.png/600px-645Landorus.png 完成
646. 抓取 https://media.52poke.com/wiki/thumb/c/c3/646Kyurem.png/600px-646Kyurem.png 完成
647. 抓取 https://media.52poke.com/wiki/thumb/5/50/647Keldeo.png/600px-647Keldeo.png 完成
648. 抓取 https://media.52poke.com/wiki/thumb/a/a3/648Meloetta.png/600px-648Meloetta.png 完成
649. 抓取 https://me

738. 抓取 https://media.52poke.com/wiki/thumb/4/4e/738Vikavolt.png/500px-738Vikavolt.png 完成
739. 抓取 https://media.52poke.com/wiki/thumb/9/98/739Crabrawler.png/500px-739Crabrawler.png 完成
740. 抓取 https://media.52poke.com/wiki/thumb/1/17/740Crabominable.png/500px-740Crabominable.png 完成
741. 抓取 https://media.52poke.com/wiki/thumb/e/ed/741Oricorio-Baile.png/600px-741Oricorio-Baile.png 完成
742. 抓取 https://media.52poke.com/wiki/thumb/f/fa/742Cutiefly.png/500px-742Cutiefly.png 完成
743. 抓取 https://media.52poke.com/wiki/thumb/e/e4/743Ribombee.png/500px-743Ribombee.png 完成
744. 抓取 https://media.52poke.com/wiki/thumb/5/51/744Rockruff.png/500px-744Rockruff.png 完成
745. 抓取 https://media.52poke.com/wiki/thumb/b/b5/745Lycanroc-Midday.png/600px-745Lycanroc-Midday.png 完成
746. 抓取 https://media.52poke.com/wiki/thumb/1/18/746Wishiwashi-Solo.png/600px-746Wishiwashi-Solo.png 完成
747. 抓取 https://media.52poke.com/wiki/thumb/d/d3/747Mareanie.png/500px-747Mareanie.png 完成
748. 抓取 https://media.52poke.com/wiki/thumb/0/06

In [38]:
img_urls

['https://media.52poke.com/wiki/thumb/2/21/001Bulbasaur.png/600px-001Bulbasaur.png',
 'https://media.52poke.com/wiki/thumb/7/73/002Ivysaur.png/600px-002Ivysaur.png',
 'https://media.52poke.com/wiki/thumb/a/ae/003Venusaur.png/600px-003Venusaur.png',
 'https://media.52poke.com/wiki/thumb/7/73/004Charmander.png/600px-004Charmander.png',
 'https://media.52poke.com/wiki/4/4a/005Charmeleon.png',
 'https://media.52poke.com/wiki/thumb/7/7e/006Charizard.png/600px-006Charizard.png',
 'https://media.52poke.com/wiki/thumb/3/39/007Squirtle.png/600px-007Squirtle.png',
 'https://media.52poke.com/wiki/0/0c/008Wartortle.png',
 'https://media.52poke.com/wiki/thumb/0/02/009Blastoise.png/600px-009Blastoise.png',
 'https://media.52poke.com/wiki/5/5d/010Caterpie.png',
 'https://media.52poke.com/wiki/thumb/c/cd/011Metapod.png/600px-011Metapod.png',
 'https://media.52poke.com/wiki/thumb/d/d1/012Butterfree.png/600px-012Butterfree.png',
 'https://media.52poke.com/wiki/thumb/d/df/013Weedle.png/600px-013Weedle.pn

**4.1将 img_urls 保存成 csv**

In [47]:
img_df = pd.DataFrame({'img_url': img_urls})

In [61]:
img_df.to_csv('pokemon-image-urls.csv', index=False)

**5.下载并保存图片**

In [51]:
# 创建文件夹路径
path = './pokemon-img/'
if not os.path.exists(path):
    os.makedirs(path)
else:
    print('文件路径已存在！')

文件路径已存在！


In [59]:
def download_img(url):
    '''
    根据 img_link 下载 Pokemon 图片
    
    params:
        path: 文件保存的文件夹路径
        url: 需下载的图片链接
    '''
    r = requests.get(url, stream=True)
    
    # 保存的文件名
    file_name = url.split('/')[-1]
    
    # 文件路径
    file_path = path+file_name
    
    # 需指定文件名，否则会出现 [Errno 21] Is a directory 错误
    with open(file_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=32):
            f.write(chunk)
        print('下载 {} 成功！'.format(file_name))

In [60]:
download_img(random.choice(img_urls))

下载 075Graveler.png 成功！
